<a href="https://colab.research.google.com/github/zoran-data/colab_notebooks/blob/main/program_offices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.5 MB/s eta 0:00:00


In [ ]:
pip install fuzzywuzzy

In [ ]:
import pandas as pd
import numpy as np
import json
import openai
import json
import pandas as pd
import requests
import time
import re
import os
from fuzzywuzzy import fuzz

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
OPENAI_BRENDAN = os.getenv("OPENAI_BRENDAN")

openai.api_key = os.getenv("OPENAI_BRENDAN")

In [ ]:

def is_token_an_acronym(token):
    tok = re.sub("[^0-9a-zA-Z]", "", token)
    out = tok[0].isupper() and tok[-1].isupper()

    return out


def get_ordinals(text):
    out = re.findall("[0-9]+(?:st| st|nd| nd|rd| rd|th| th)", text)
    return out


def process_items(items):
    if "\n-" in items:
        out = items.split("\n-")
    else:
        out = items.split(", ")

    out = [u.split("\n") for u in out]

    out = [v for u in out for v in u if len(v) > 3]
    out = [u.replace("•", "").replace("\t", "") for u in out]

    return out



def custom_model_find_in_list(item, program_offices_of_department_list, min_score, with_lower):
    correct_po = str()
    #if with_lower:
    #    item_ = re.sub("[^0-9a-zA-Z]", "", item.lower())
    #else:
    #    item_ = re.sub("[^0-9a-zA-Z]", "", item)

    min_lev = min_score
    matching_name = str()

    for po in program_offices_of_department_list:
        main_name = po["official_name"]
        all_names = po["all_names"]

        for name in all_names:
           # if with_lower:
           #     name_ = re.sub("[^0-9a-zA-Z]", "", name.lower())
           # else:
           #     name_ = re.sub("[^0-9a-zA-Z]", "", name)
            if with_lower:
                new_val = fuzz.ratio(item.lower(), name.lower())
            else:
                new_val = fuzz.ratio(str(item), str(name))

            #print("item - name - score : ", item, " - ", name, " - ", new_val)
            if  new_val >= min_lev:

                min_lev = new_val
                matching_name = main_name



    item_ordinals = get_ordinals(item)
    match_name_ordinals = get_ordinals(matching_name)

    if len(item_ordinals) * len(match_name_ordinals) >0:
        if not item_ordinals[0] == match_name_ordinals[0]:
            print("Remove because ordinals not matching")
            matching_name = str()


    return matching_name

def get_acronyms(text):
    out = re.findall(r'\b[A-Z](?:[&.]?[A-Z])+\b',text)
    return out

def is_acronym(text):
    score = (sum(1 for c in text if c.isupper()))/len(text)
    if score > 0.5:
        out = True
    else:
        out = False
    return out

def custom_model_find_in_list_with_acronyms(item, program_offices_of_department_list):
    matching_name = str()
    for po in program_offices_of_department_list:
        main_name = po["official_name"]
        gpt3_acronym = po["gpt3_acronym"]
        initial_acronym = po["initial_acronym"]

        if is_acronym(item):
            if str(initial_acronym) == item or str(gpt3_acronym) == item:
                matching_name = main_name

        else:
            item_acronyms = get_acronyms(item)
            if str(initial_acronym) in item_acronyms or str(gpt3_acronym) in item_acronyms:
                matching_name = main_name

    return matching_name


def screen_text_for_program_offices(text, program_offices_list, min_score):
    out = list()
    for program in program_offices_list:
        if fuzz.partial_ratio(str(text), str(program)) >= min_score:
            out += [program]
    return out


def build_list_of_program_offices(program_offices_of_department):
    out = list()
    for i in range(program_offices_of_department.shape[0]):
        official_name = program_offices_of_department.loc[:, "official_program_office_name"].iloc[i]
        initial_acronym = program_offices_of_department.loc[:, "program_office_initial_acronym"].iloc[i]
        gpt3_acronym = program_offices_of_department.loc[:, "program_office_gpt3_acronym"].iloc[i]
        departement = program_offices_of_department.loc[:, "department"].iloc[i]
        record_id = program_offices_of_department.loc[:, "airtable_record_ID"].iloc[i]
        all_names = json.loads(program_offices_of_department.loc[:, "program_offices_name_common_variations_clean"].iloc[i]) + [official_name] + [program_offices_of_department.loc[:, "program_office_initial_name"].iloc[i]]
        all_names = list(set(all_names))
        out +=  [{
            "official_name": official_name,
            "department": departement,
            "record_id": record_id,
            "gpt3_acronym": gpt3_acronym,
            "initial_acronym": initial_acronym,
            "all_names": all_names
            }]
    return out


def build_agency_to_dep_dict(agency_file):
    data = pd.read_excel(agency_file)
    out = dict()
    for i in range(data.shape[0]):
        out[data.loc[:, "Name"].iloc[i]] = data.loc[:, "Department Parent"].iloc[i]
    return out

if False:
    def gpt3(text):
        if True:
            time.sleep(3)

            headers = {
                'Content-Type': 'application/json',
                'Authorization': f"Bearer {OPENAI_BRENDAN}",
            }

            json_data = {
            'model': 'text-davinci-002',
            'prompt': "Extract program offices from this text:\n\n" + text ,
            'temperature': 0.0,
            'max_tokens': 30,
            'top_p': 1,
            'frequency_penalty': 0,
            'presence_penalty': 0,
            }

            #print(json_data)
            response = requests.post('https://api.openai.com/v1/completions', headers=headers, json=json_data)
            response = json.loads(response.text)
            out = response["choices"][0]["text"]
            finish_reason = response["choices"][0]["finish_reason"]
        else:
            out = str()
            finish_reason = str()
        return out, finish_reason

def get_completion(prompt, model="gpt-3.5-turbo", max_tokens=100):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
        max_tokens=max_tokens
    )
    output_text = response.choices[0].message["content"]

    return output_text


In [ ]:
examples = [
"""- Led daily operations for 24-person team leading high-vis US government reviews of 166 detainees at Guantanamo Bay, Cuba.
- Developed 1st-ever policy procedures, risk analysis, and executed strategic reviews for Secretary of Defense and National Security Council (NSC).""",
"""- Special Assignment - Secretary of Defense directed Independent Review of the Washington Navy Yard shooting.

- Identified, interviewed and selected 27 member team to analyze, assess and complete report on time and within directed parameters.

- Report recommendations adopted by Secretary of Defense and supported by the President and resulted in implementation of personnel security safeguards
 across the Defense Department and US Government.  To include continuous evaluation process.

- Orchestrated cross-jurisdictional fact finding at DoD, Federal, State, and Local Law Enforcement, Medical Community, and Defense Homeland Security.""",
"""I was advanced through various roles and selected to- and steered Headquarters Airforce’s (HAF) IT enterprise strategy for the air service branch of the U.S. armed forces. In this role, I provided oversight ensuring organizational information assurance and IT services while also serving as Lead Advisory to mission-critical initiatives, and contracts including the negotiating and managing of HAF’s 1st IT support agreement.

I was recognized with the Exceptional Civilian Service Award (Highest Civilian Honor Granted by Secretary of the Air Force);
and the Meritorious Civilian Service Medal as Director, 75th Communications and IT Directorate role. Additionally,
I held the prestigious Board leadership as Chair of the Pentagon Architecture Configuration Control Board,
establishing the IT strategy and protocol for common services, which has served as DoD’s standard for over a decade.""",
"""Currently serving as the Government Purchase Card Program Manager Cannon AFB and the 27th Special Operation Wing with oversight of over $25 M in annual operational expenditures and functioning the direct point of contact for unit commanders and program participants.

Primary liaison to U.S. Bank and the 27th Special Operations Wing to effectively resolve issues, maintain vendor relations, conduct audits, and ensure program compliance.

Daily responsibilities is to act as the functional bridge between the fiance unit and the contracting squadron at the micro-purchase level and in scope with GSA Smart-pay.

Acting as a Inspector General Wing Inspection Team member; providing support to Special Operation Wing Inspector General's inspections and audits.  Oversaw the Inspector General's installation wide inspections of unit Government Purchase Card programs.""",
]

In [ ]:
# program_offices_file = "./program_offices_with_new_acronyms_dep_acronyms_new_acronyms_official_names_variations_clean.xlsx"
agency_file = "./departments_agencies.xlsx"
input_file = "../Bases/gov_experience_large.csv"
program_offices_file = "program_offices_all.xlsx"

data = pd.read_csv(input_file)
program_offices = pd.read_excel(program_offices_file)

In [ ]:
data.head()

,Description
0,Program manager of individual equipment items ...
1,"Responsible for research, development, acquisi..."
2,"Directed human resource management, training, ..."
3,Senior military reserve leader and advisor to ...
4,Director of the safety function for an entire ...


In [ ]:
program_offices.head()

,airtable_record_ID,department,department_without_acronym,department_acronym,program_office_initial_acronym,program_office_initial_name,ACRONYM,URL,BASE TEXT,LOCATION,...,Gov Experience,Industry Experience,Associated Contracts,Projects,Profile Signups,Firms : All,Associated Program Office,Experience Import,INITIAL WITHOUT ACRONYM,program_offices_name_common_variations
0,recm7UjZwaHet3Q1n,Department of the Army (USA),Department of the Army,USA,NaN,U.S. Army Engineering and Support Center Hunts...,USAESCH,https://www.huntsvillecenter.army.mil/,"Huntsville, Alabama","Huntsville, AL",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U.S. Army Engineering and Support Center Hunts...,"['U.S. Army Engineering and Support Center, Hu..."
1,recBGy6S9Et9ZIAqj,State Department,State Department,NaN,NaN,Cyber and International Communications and Inf...,CICIP,https://www.state.gov/cyber-international-comm...,The Pentagon,"Washington, D.C.",...,NaN,NaN,NaN,NaN,NaN,NaN,169,NaN,Cyber and International Communications and Inf...,"['Office of the Coordinator for Cyber Issues',..."
2,reciVAKmkU8b8dcjk,Department of Defense (DoD),Department of Defense,DoD,NaN,DCMA Louisiana/East Texas,DCMA LA/ET,http://www.dcmalaetx.com/,http://www.dcmalaetx.com/,"Baton Rouge, LA",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Defense Contract Management Agency Louisiana/E...,"['DCMA-LA/ET', 'Agency', 'DCMA']"
3,recAA38tRlo2gEvd9,Trade and Development Agency (USTDA),Trade and Development Agency,USTDA,NaN,United States Trade & Development Agency,USTDA,https://www.usaid.gov/,NaN,"Washington, D.C.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States Trade & Development Agency,"['Agency', 'USTDA', 'United States Trade and D..."
4,recG7u7iskUmxhwh0,Department of Homeland Security (DHS),Department of Homeland Security,DHS,NaN,Commanding Officer U.S. Coast Guard Training C...,COTC,https://www.petaluma.uscgaux.info/,U.S. Coast Guard Training Center Petaluma,"Petaluma, CA",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Commanding Officer U.S. Coast Guard Training C...,"['Captain David A. Bordelon', 'Commanding Offi..."


In [ ]:
data.shape, program_offices.shape

((638, 1), (18006, 39))

In [ ]:
text = examples[2]
print(text)

I was advanced through various roles and selected to- and steered Headquarters Airforce’s (HAF) IT enterprise strategy for the air service branch of the U.S. armed forces. In this role, I provided oversight ensuring organizational information assurance and IT services while also serving as Lead Advisory to mission-critical initiatives, and contracts including the negotiating and managing of HAF’s 1st IT support agreement.
 
I was recognized with the Exceptional Civilian Service Award (Highest Civilian Honor Granted by Secretary of the Air Force); 
and the Meritorious Civilian Service Medal as Director, 75th Communications and IT Directorate role. Additionally, 
I held the prestigious Board leadership as Chair of the Pentagon Architecture Configuration Control Board, 
establishing the IT strategy and protocol for common services, which has served as DoD’s standard for over a decade.


In [ ]:
all_departments = program_offices["department"].tolist()

In [ ]:
prompt = f"""
Extract the program offices from the text delimited by <>:
<{text}>

Output the results as a list delimited with the | character.
If there is no program office found, return "none"
"""

items = get_completion(prompt, model="gpt-3.5-turbo", max_tokens=200)

In [ ]:
program_offices_candidates = items.split("|")

In [ ]:
program_offices_candidates

['HAF',
 '75th Communications and IT Directorate',
 'Pentagon Architecture Configuration Control Board']

In [ ]:
program_offices[program_offices.department_acronym.isin(["HAF"])]

,airtable_record_ID,department,department_without_acronym,department_acronym,program_office_initial_acronym,program_office_initial_name,ACRONYM,URL,BASE TEXT,LOCATION,...,Gov Experience,Industry Experience,Associated Contracts,Projects,Profile Signups,Firms : All,Associated Program Office,Experience Import,INITIAL WITHOUT ACRONYM,program_offices_name_common_variations


In [ ]:
program_offices.head()

,airtable_record_ID,department,department_without_acronym,department_acronym,program_office_initial_acronym,program_office_initial_name,ACRONYM,URL,BASE TEXT,LOCATION,...,Gov Experience,Industry Experience,Associated Contracts,Projects,Profile Signups,Firms : All,Associated Program Office,Experience Import,INITIAL WITHOUT ACRONYM,program_offices_name_common_variations
0,recm7UjZwaHet3Q1n,Department of the Army (USA),Department of the Army,USA,NaN,U.S. Army Engineering and Support Center Hunts...,USAESCH,https://www.huntsvillecenter.army.mil/,"Huntsville, Alabama","Huntsville, AL",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U.S. Army Engineering and Support Center Hunts...,"['U.S. Army Engineering and Support Center, Hu..."
1,recBGy6S9Et9ZIAqj,State Department,State Department,NaN,NaN,Cyber and International Communications and Inf...,CICIP,https://www.state.gov/cyber-international-comm...,The Pentagon,"Washington, D.C.",...,NaN,NaN,NaN,NaN,NaN,NaN,169,NaN,Cyber and International Communications and Inf...,"['Office of the Coordinator for Cyber Issues',..."
2,reciVAKmkU8b8dcjk,Department of Defense (DoD),Department of Defense,DoD,NaN,DCMA Louisiana/East Texas,DCMA LA/ET,http://www.dcmalaetx.com/,http://www.dcmalaetx.com/,"Baton Rouge, LA",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Defense Contract Management Agency Louisiana/E...,"['DCMA-LA/ET', 'Agency', 'DCMA']"
3,recAA38tRlo2gEvd9,Trade and Development Agency (USTDA),Trade and Development Agency,USTDA,NaN,United States Trade & Development Agency,USTDA,https://www.usaid.gov/,NaN,"Washington, D.C.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States Trade & Development Agency,"['Agency', 'USTDA', 'United States Trade and D..."
4,recG7u7iskUmxhwh0,Department of Homeland Security (DHS),Department of Homeland Security,DHS,NaN,Commanding Officer U.S. Coast Guard Training C...,COTC,https://www.petaluma.uscgaux.info/,U.S. Coast Guard Training Center Petaluma,"Petaluma, CA",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Commanding Officer U.S. Coast Guard Training C...,"['Captain David A. Bordelon', 'Commanding Offi..."


In [ ]:
program_offices.sample(3).department_without_acronym.to_list()

['Department of the Interior',
 'Department of the Navy',
 'Department of the Interior']

In [ ]:
        ############# 1. check program offices with custom fuzzywuzzy
        min_score = 85
        with_lower = False
        matched_program_offices_withoutlower = [custom_model_find_in_list(u, program_offices_of_department_list, min_score, with_lower) for u in program_offices_candidates]
        matched_program_offices_withoutlower = [u for u in matched_program_offices_withoutlower if len(u) > 0]
        matched_program_offices_withoutlower = list(set(matched_program_offices_withoutlower))
        print("Number of program offices mateched without lower : ", len(matched_program_offices_withoutlower))
        # check program offices with custom model #############
    else:
        matched_program_offices_withoutlower = list()
    print("Programm offices matched gpt3 : ", matched_program_offices_withoutlower)

In [ ]:
print("#################    ", s)
text = data.loc[:, "Description"].iloc[s]
departments = str(data.loc[:, "Department"].iloc[s])
gov_exp_departments = [u for u in departments.split(",") if not u =="nan"]
nb_departments = len(gov_exp_departments)
print("Input departments: ",departments )
print("Number of departments : ", nb_departments)
print("Gov experience departments 1: ", gov_exp_departments)


############# add agencies to gov experience departments
all_departments = program_offices["department"].tolist()

for p, depart in enumerate(gov_exp_departments):
    if not depart in all_departments and depart in agency_to_dep_dict:
        gov_exp_departments[p] = agency_to_dep_dict[depart]
print("Gov experience departments 2: ", gov_exp_departments)

############# Get program_offices_candidates from text
items, finish_reason = gpt3(text)
program_offices_candidates = process_items(items)
# Get program_offices_candidates from text   #############

print("Program offices candidates : ", program_offices_candidates)

#################     0
Input departments:  
Number of departments :  1
Gov experience departments 1:  ['']


In [ ]:
print(text)

Program manager of individual equipment items to include body armor and medical equipment programs/projects totaling over $50M per year. 

Acquisition and sustainment enabler for joint portfolio that includes vehicles, weapons, visual augmentation systems, and electronic countermeasures.


In [ ]:
# agency_to_dep_dict

In [ ]:
repeat = 0

data["associated_departements"] = str()
data["gpt3_before_processing"] = str()
data["gpt3_finish_reason"] = str()
data["gpt3_after_processing"] = str()
data["nb_departments_found_in_departments_list"] = str()
data["departments_not_found"] = str()
data["1. search in dep program offices with lower"] = str()
data["1. search in dep program offices without lower"] = str()
data["2. search in dep program offices with acronyms"] = str()
data['Department'] = ""
data["all"] = str()


agency_to_dep_dict = build_agency_to_dep_dict(agency_file)
print("Number of agencies : ", len(agency_to_dep_dict))

for s in range(data.shape[0]):
    departments_not_found = str()

    print("#################    ", s)
    text = data.loc[:, "Description"].iloc[s]
    departments = str(data.loc[:, "Department"].iloc[s])
    gov_exp_departments = [u for u in departments.split(",") if not u =="nan"]
    nb_departments = len(gov_exp_departments)
    print("Input departments: ",departments )
    print("Number of departments : ", nb_departments)
    print("Gov experience departments 1: ", gov_exp_departments)


    ############# add agencies to gov experience departments
    all_departments = program_offices["department"].tolist()

    for p, depart in enumerate(gov_exp_departments):
        if not depart in all_departments and depart in agency_to_dep_dict:
            gov_exp_departments[p] = agency_to_dep_dict[depart]
    print("Gov experience departments 2: ", gov_exp_departments)
    # add agencies to gov experience departments #############


    ############# log in DF
    data.loc[:, "associated_departements"].iloc[s] = str(nb_departments)
    # log in DF  #############

    if nb_departments > 0:

        ############# Get program_offices_candidates from text
        items, finish_reason = gpt3(text)
        program_offices_candidates = process_items(items)
        # Get program_offices_candidates from text   #############

        print("Program offices candidates : ", program_offices_candidates)

        ############# log in DF
        data.loc[:, "gpt3_before_processing"].iloc[s] = items
        data.loc[:, "gpt3_after_processing"].iloc[s] = program_offices_candidates
        data.loc[:, "gpt3_finish_reason"].iloc[s] = finish_reason
        # log in DF  #############

        ############# Get program offices fixed list from departments
        program_offices_of_department = program_offices[program_offices["department"].isin(gov_exp_departments)]

        print("Nb Program offices candidates for departments: ", program_offices_of_department.shape[0] )

        program_offices_of_department_list = build_list_of_program_offices(program_offices_of_department)
        # Get program offices fixed list from departments   #############


        ############# log in DF nb_departments_found_in_departments_list_1
        nb_departments_found_in_departments_list = len(list(set(program_offices_of_department["department"].tolist())))
        data.loc[:, "nb_departments_found_in_departments_list"].iloc[s] = str(nb_departments_found_in_departments_list)
        print("Number of departments identified in full program offices list :", nb_departments_found_in_departments_list)
        # log in DF nb_departments_found_in_departments_list_1 #############

        ############# log in DF departments_not_found
        for dep in gov_exp_departments:
            if not dep in list(set(program_offices_of_department["department"].tolist())) and not dep == "nan":
                if len(departments_not_found) > 0:
                    departments_not_found += ", " + dep
                else:
                    departments_not_found = dep
        data.loc[:, "departments_not_found"].iloc[s] = departments_not_found
        print("Departments not found : ", departments_not_found)
        # log in DF departments_not_found #############



        """

        ############# get program offices to check
        if int(nb_departments_found_in_departments_list) > 0:
            program_offices_to_check = get_program_offices_list_from_dict(program_offices_of_department_dict)
        else:
            program_offices_to_check = list()
        print("Number of program offices considered for the search : {}".format(str(len(program_offices_to_check))))
        # get program offices to check #############

        """

        ############# 1. check program offices with custom fuzzywuzzy
        min_score = 85
        with_lower = False
        matched_program_offices_withoutlower = [custom_model_find_in_list(u, program_offices_of_department_list, min_score, with_lower) for u in program_offices_candidates]
        matched_program_offices_withoutlower = [u for u in matched_program_offices_withoutlower if len(u) > 0]
        matched_program_offices_withoutlower = list(set(matched_program_offices_withoutlower))
        print("Number of program offices mateched without lower : ", len(matched_program_offices_withoutlower))
        # check program offices with custom model #############
    else:
        matched_program_offices_withoutlower = list()
    print("Programm offices matched gpt3 : ", matched_program_offices_withoutlower)
    matched_program_offices_with_acronym = [custom_model_find_in_list_with_acronyms(u, program_offices_of_department_list) for u in program_offices_candidates]
    matched_program_offices_with_acronym = list(set(matched_program_offices_with_acronym))
    matched_program_offices_with_acronym = [u for u in matched_program_offices_with_acronym if len(u) >0]

    #matched_program_offices_with_acronym_in_text = get_po_from_acronym_in_text(text, program_offices_of_department_list)
    data.loc[:, "1. search in dep program offices without lower"].iloc[s] = json.dumps(matched_program_offices_withoutlower)
    data.loc[:, "2. search in dep program offices with acronyms"].iloc[s] = json.dumps(matched_program_offices_with_acronym)
    all_ = list(set(matched_program_offices_with_acronym + matched_program_offices_withoutlower))
    all_ = [u.replace("\n", "") for u in all_]
    data.loc[:, "all"].iloc[s] = json.dumps(all_)
    print("All : ", all_)


Number of agencies :  204
#################     0
Input departments:  
Number of departments :  1
Gov experience departments 1:  ['']
Gov experience departments 2:  ['']
Program offices candidates :  ['The program offices are:', 'Program manager of individual equipment items', 'Acquisition and sustainment enabler for joint portfolio']
Nb Program offices candidates for departments:  0
Number of departments identified in full program offices list : 0
Departments not found :  
Number of program offices mateched without lower :  0
Programm offices matched gpt3 :  []
All :  []
#################     1
Input departments:  
Number of departments :  1
Gov experience departments 1:  ['']
Gov experience departments 2:  ['']
Program offices candidates :  ['SOCOM mission support and intelligence programs', 'DCGS-SOF', 'SOFPREP', 'Signals Intelligence Systems', 'and Integrated Survey Program']
Nb Program offices candidates for departments:  0
Number of departments identified in full program offices 

KeyboardInterrupt: 